## Data Prep


### Load Data

In [1]:
# Classic Python data science Starter pack

from pandas import DataFrame
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.formula.api as smf # linear modeling
from sklearn.model_selection import train_test_split # Split into train test data

features = pd.read_csv('data/dengue_features_train.csv')
labels = pd.read_csv('data/dengue_labels_train.csv')

### Wrangle Data

In [2]:
# Merge Train features and labels into one table
# result = pd.merge(train_features, train_labels, on=['weekofyear','year', 'city'])

# Drop Extra features
features = features.drop(columns=[ 'station_max_temp_c','station_precip_mm', 'station_diur_temp_rng_c', 'precipitation_amt_mm'])

# Fill missing values with ffill
features.fillna(method='ffill', inplace=True)
labels.fillna(method='ffill', inplace=True)

In [3]:
labels.head()

,city,year,weekofyear,total_cases
0,sj,1990,18,4
1,sj,1990,19,5
2,sj,1990,20,4
3,sj,1990,21,3
4,sj,1990,22,6


### Add columns and split data by city
##### 0 - Iquitos
##### 1 - San Juan

In [4]:
# add column lag cases
def add_lag_cases(a, n):
    a= a.copy()
    
    # set columnm name
    name =  "lag_cases_" + str(n)
    
    #save first n values of total case
    last_n = []
    for i in range(len(a) - n,len(a)):
        last_n.append(a.iloc[i, a.columns.get_loc('total_cases')] )
        
    last_val = sum(last_n)/len(last_n)
    # shift columns  
    a[name] = a.total_cases.shift(-n)

    # add last values
    for i in range(len(a) - n,len(a)):
        a[name][i] = last_val
    return a


def subset_by_city(city, df):
    return df[df.city == city]



In [5]:
# subsets data by CITY 
features_by_city = [subset_by_city(c, features).copy().reset_index(drop=True) for c in ['iq','sj']]
labels_by_city = [subset_by_city(c, labels).copy().reset_index(drop=True) for c in ['iq','sj']]


# add lag cases to labels
# add 4 weeks lag
labels_by_city[0] = add_lag_cases(labels_by_city[0], 4)
labels_by_city[1] = add_lag_cases(labels_by_city[1], 4)

# add 8 weeks lag
labels_by_city[0] = add_lag_cases(labels_by_city[0], 8)
labels_by_city[1] = add_lag_cases(labels_by_city[1], 8)

# add 12 weeks lag
labels_by_city[0] = add_lag_cases(labels_by_city[0], 12)
labels_by_city[1] = add_lag_cases(labels_by_city[1], 12)


# add mean vegetation index column
features_by_city[0]['mean_vegetation_index'] = (features_by_city[0]['ndvi_se'] + features_by_city[0]['ndvi_sw'] + features_by_city[0]['ndvi_ne'] + features_by_city[0]['ndvi_nw']) / 4
features_by_city[1]['mean_vegetation_index'] = (features_by_city[1]['ndvi_se'] + features_by_city[1]['ndvi_sw'] + features_by_city[1]['ndvi_ne'] + features_by_city[1]['ndvi_nw']) / 4

# Drop vegetation columns
features_by_city[0] = features_by_city[0].drop(columns=['ndvi_ne', 'ndvi_nw', 'ndvi_se','ndvi_sw'])
features_by_city[1] = features_by_city[1].drop(columns=['ndvi_ne', 'ndvi_nw', 'ndvi_se','ndvi_sw'])


/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [6]:
f = ['reanalysis_specific_humidity_g_per_kg', 
                 'reanalysis_dew_point_temp_k', 
                 'station_avg_temp_c', 
                 'station_min_temp_c',
                 'mean_vegetation_index']


features_by_city[0] = features_by_city[0][f]
features_by_city[1] = features_by_city[1][f]

### Split Test, Train data for each city

In [7]:
# Iquitos
I_train_features, I_test_features, I_train_outcome, I_test_outcome = train_test_split(
    features_by_city[0],
    labels_by_city[0],
    test_size = 0.3,
    random_state = 11
)

In [8]:
# San Juan
sj_train_features, sj_test_features, sj_train_outcome, sj_test_outcome = train_test_split(
    features_by_city[1],
    labels_by_city[1],
    test_size = 0.3,
    random_state = 11
)

To-do
* Select appropriate columns
* 

## Implementing Models

In [9]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import MinMaxScaler       # scaling data
from sklearn.neighbors import KNeighborsRegressor    # regressor
from sklearn.model_selection import GridSearchCV     # for grid search
from sklearn.pipeline import make_pipeline  
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures()# for making pipelines
scaler = MinMaxScaler()
knn_reg = KNeighborsRegressor()
from sklearn.linear_model import LinearRegression
linear_reg = LinearRegression()
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import RidgeCV


#### KNN Regressor

In [167]:
# Define a pipeline that includes the polynomial transformation
pipe = make_pipeline(poly, scaler, knn_reg)

# Define a grid to search through (including the degree of polynomial)
param_grid = {'polynomialfeatures__degree':range(1, 3), 
              'kneighborsregressor__n_neighbors':range(1, 5), 
              'kneighborsregressor__weights':["uniform", "distance"]}

# Perform a  grid search of your pipeline
grid = GridSearchCV(pipe, param_grid, scoring="neg_mean_absolute_error")

grid.fit(I_train_features, I_train_outcome.lag_cases_12)
print(grid.score(I_test_features, I_test_outcome.total_cases))
print(grid.best_params_)


grid.fit(sj_train_features, sj_train_outcome.total_cases)
print(grid.score(sj_test_features, sj_test_outcome.total_cases))
print(grid.best_params_)



-7.01368272923
{'kneighborsregressor__n_neighbors': 4, 'kneighborsregressor__weights': 'distance', 'polynomialfeatures__degree': 1}
-30.4887307236
{'kneighborsregressor__n_neighbors': 3, 'kneighborsregressor__weights': 'uniform', 'polynomialfeatures__degree': 1}


In [141]:
grid.best_params_

{'kneighborsregressor__n_neighbors': 3,
 'kneighborsregressor__weights': 'uniform',
 'polynomialfeatures__degree': 1}

In [115]:
labels.head()

,city,year,weekofyear,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,...,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm
0,sj,2008,18,2008-04-29,-0.0189,-0.018900,0.102729,0.091200,78.60,298.492857,...,25.37,78.781429,78.60,15.918571,3.128571,26.528571,7.057143,33.3,21.7,75.2
1,sj,2008,19,2008-05-06,-0.0180,-0.012400,0.082043,0.072314,12.56,298.475714,...,21.83,78.230000,12.56,15.791429,2.571429,26.071429,5.557143,30.0,22.2,34.3
2,sj,2008,20,2008-05-13,-0.0015,NaN,0.151083,0.091529,3.66,299.455714,...,4.12,78.270000,3.66,16.674286,4.428571,27.928571,7.785714,32.8,22.8,3.0
3,sj,2008,21,2008-05-20,NaN,-0.019867,0.124329,0.125686,0.00,299.690000,...,2.20,73.015714,0.00,15.775714,4.342857,28.057143,6.271429,33.3,24.4,0.3
4,sj,2008,22,2008-05-27,0.0568,0.039833,0.062267,0.075914,0.76,299.780000,...,4.36,74.084286,0.76,16.137143,3.542857,27.614286,7.085714,33.3,23.3,84.1


Random Test

In [176]:
ridge = RidgeCV()


# Define a pipeline that includes the polynomial transformation
pipe = make_pipeline(poly, scaler, RidgeCV())

# Define a grid to search through (including the degree of polynomial)
param_grid = {'polynomialfeatures__degree':range(1, 3) }

# Perform a  grid search of your pipeline
grid = GridSearchCV(pipe, param_grid, scoring="neg_mean_absolute_error")

grid.fit(I_train_features, I_train_outcome.lag_cases_12)
print(grid.score(I_test_features, I_test_outcome.total_cases))
print(grid.best_params_)


grid.fit(sj_train_features, sj_train_outcome.total_cases)
print(grid.score(sj_test_features, sj_test_outcome.total_cases))
print(grid.best_params_)



-6.67216834615
{'polynomialfeatures__degree': 2}
-27.8614821849
{'polynomialfeatures__degree': 2}


In [147]:
ridge.get_params().keys()

dict_keys(['alphas', 'cv', 'fit_intercept', 'gcv_mode', 'normalize', 'scoring', 'store_cv_values'])

In [169]:
grid

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(steps=[('polynomialfeatures', PolynomialFeatures(degree=2, include_bias=True, interaction_only=False)), ('minmaxscaler', MinMaxScaler(copy=True, feature_range=(0, 1))), ('ridgecv', RidgeCV(alphas=(0.1, 1.0, 10.0), cv=None, fit_intercept=True, gcv_mode=None,
    normalize=False, scoring=None, store_cv_values=False))]),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'polynomialfeatures__degree': range(1, 3), 'ridgecv__alphas': [0.001, 0.01, 0.1, 1]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='neg_mean_absolute_error', verbose=0)

### Random Forest 

In [178]:
from sklearn.ensemble import RandomForestRegressor
random_forest = RandomForestRegressor()

In [ ]:


# Define a pipeline that includes the polynomial transformation
pipe = make_pipeline(poly, scaler, random_forest)

# Define a grid to search through (including the degree of polynomial)
param_grid = {'polynomialfeatures__degree':range(1, 3),
             'randomforestregressor__max_depth':range(1,80),
             'randomforestregressor__bootstrap': [True, False],
             'randomforestregressor__max_depth': [10, 30,  50,  70,  90, None],
             'randomforestregressor__max_features': ['auto', 'sqrt'],
             'randomforestregressor__min_samples_leaf': [1, 2, 4],
             'randomforestregressor__min_samples_split': [2, 5, 10],
             'randomforestregressor__n_estimators': [200, 600,  1000,  1400,  1800]}

# Perform a  grid search of your pipeline
grid = GridSearchCV(pipe, param_grid, scoring="neg_mean_absolute_error", verbose=10)

grid.fit(I_train_features, I_train_outcome.lag_cases_12)
print(grid.score(I_test_features, I_test_outcome.total_cases))
print(grid.best_params_)


grid.fit(sj_train_features, sj_train_outcome.total_cases)
print(grid.score(sj_test_features, sj_test_outcome.total_cases))
print(grid.best_params_)


Fitting 3 folds for each of 2160 candidates, totalling 6480 fits
[CV] polynomialfeatures__degree=1, randomforestregressor__bootstrap=True, randomforestregressor__max_depth=10, randomforestregressor__max_features=auto, randomforestregressor__min_samples_leaf=1, randomforestregressor__min_samples_split=2, randomforestregressor__n_estimators=200 
[CV]  polynomialfeatures__degree=1, randomforestregressor__bootstrap=True, randomforestregressor__max_depth=10, randomforestregressor__max_features=auto, randomforestregressor__min_samples_leaf=1, randomforestregressor__min_samples_split=2, randomforestregressor__n_estimators=200, score=-7.535366, total=   0.3s
[CV] polynomialfeatures__degree=1, randomforestregressor__bootstrap=True, randomforestregressor__max_depth=10, randomforestregressor__max_features=auto, randomforestregressor__min_samples_leaf=1, randomforestregressor__min_samples_split=2, randomforestregressor__n_estimators=200 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s remaining:    0.0s


[CV]  polynomialfeatures__degree=1, randomforestregressor__bootstrap=True, randomforestregressor__max_depth=10, randomforestregressor__max_features=auto, randomforestregressor__min_samples_leaf=1, randomforestregressor__min_samples_split=2, randomforestregressor__n_estimators=200, score=-6.917140, total=   0.3s
[CV] polynomialfeatures__degree=1, randomforestregressor__bootstrap=True, randomforestregressor__max_depth=10, randomforestregressor__max_features=auto, randomforestregressor__min_samples_leaf=1, randomforestregressor__min_samples_split=2, randomforestregressor__n_estimators=200 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.7s remaining:    0.0s


[CV]  polynomialfeatures__degree=1, randomforestregressor__bootstrap=True, randomforestregressor__max_depth=10, randomforestregressor__max_features=auto, randomforestregressor__min_samples_leaf=1, randomforestregressor__min_samples_split=2, randomforestregressor__n_estimators=200, score=-8.333697, total=   0.3s
[CV] polynomialfeatures__degree=1, randomforestregressor__bootstrap=True, randomforestregressor__max_depth=10, randomforestregressor__max_features=auto, randomforestregressor__min_samples_leaf=1, randomforestregressor__min_samples_split=2, randomforestregressor__n_estimators=600 


...

[CV]  polynomialfeatures__degree=2, randomforestregressor__bootstrap=False, randomforestregressor__max_depth=30, randomforestregressor__max_features=sqrt, randomforestregressor__min_samples_leaf=2, randomforestregressor__min_samples_split=10, randomforestregressor__n_estimators=600, score=-7.588883, total=   0.8s
[CV] polynomialfeatures__degree=2, randomforestregressor__bootstrap=False, randomforestregressor__max_depth=30, randomforestregressor__max_features=sqrt, randomforestregressor__min_samples_leaf=2, randomforestregressor__min_samples_split=10, randomforestregressor__n_estimators=600 
[CV]  polynomialfeatures__degree=2, randomforestregressor__bootstrap=False, randomforestregressor__max_depth=30, randomforestregressor__max_features=sqrt, randomforestregressor__min_samples_leaf=2, randomforestregressor__min_samples_split=10, randomforestregressor__n_estimators=600, score=-7.057434, total=   0.8s
[CV] polynomialfeatures__degree=2, randomforestregressor__bootstrap=False, randomforest

[CV]  polynomialfeatures__degree=2, randomforestregressor__bootstrap=False, randomforestregressor__max_depth=30, randomforestregressor__max_features=sqrt, randomforestregressor__min_samples_leaf=4, randomforestregressor__min_samples_split=2, randomforestregressor__n_estimators=200, score=-7.631907, total=   0.3s
[CV] polynomialfeatures__degree=2, randomforestregressor__bootstrap=False, randomforestregressor__max_depth=30, randomforestregressor__max_features=sqrt, randomforestregressor__min_samples_leaf=4, randomforestregressor__min_samples_split=2, randomforestregressor__n_estimators=600 
[CV]  polynomialfeatures__degree=2, randomforestregressor__bootstrap=False, randomforestregressor__max_depth=30, randomforestregressor__max_features=sqrt, randomforestregressor__min_samples_leaf=4, randomforestregressor__min_samples_split=2, randomforestregressor__n_estimators=600, score=-7.550091, total=   0.8s
[CV] polynomialfeatures__degree=2, randomforestregressor__bootstrap=False, randomforestreg

[CV]  polynomialfeatures__degree=2, randomforestregressor__bootstrap=False, randomforestregressor__max_depth=30, randomforestregressor__max_features=sqrt, randomforestregressor__min_samples_leaf=4, randomforestregressor__min_samples_split=5, randomforestregressor__n_estimators=200, score=-7.018972, total=   0.3s
[CV] polynomialfeatures__degree=2, randomforestregressor__bootstrap=False, randomforestregressor__max_depth=30, randomforestregressor__max_features=sqrt, randomforestregressor__min_samples_leaf=4, randomforestregressor__min_samples_split=5, randomforestregressor__n_estimators=200 
[CV]  polynomialfeatures__degree=2, randomforestregressor__bootstrap=False, randomforestregressor__max_depth=30, randomforestregressor__max_features=sqrt, randomforestregressor__min_samples_leaf=4, randomforestregressor__min_samples_split=5, randomforestregressor__n_estimators=200, score=-7.674726, total=   0.3s
[CV] polynomialfeatures__degree=2, randomforestregressor__bootstrap=False, randomforestreg

[CV]  polynomialfeatures__degree=2, randomforestregressor__bootstrap=False, randomforestregressor__max_depth=30, randomforestregressor__max_features=sqrt, randomforestregressor__min_samples_leaf=4, randomforestregressor__min_samples_split=10, randomforestregressor__n_estimators=200, score=-7.489039, total=   0.3s
[CV] polynomialfeatures__degree=2, randomforestregressor__bootstrap=False, randomforestregressor__max_depth=30, randomforestregressor__max_features=sqrt, randomforestregressor__min_samples_leaf=4, randomforestregressor__min_samples_split=10, randomforestregressor__n_estimators=200 
[CV]  polynomialfeatures__degree=2, randomforestregressor__bootstrap=False, randomforestregressor__max_depth=30, randomforestregressor__max_features=sqrt, randomforestregressor__min_samples_leaf=4, randomforestregressor__min_samples_split=10, randomforestregressor__n_estimators=200, score=-6.938079, total=   0.3s
[CV] polynomialfeatures__degree=2, randomforestregressor__bootstrap=False, randomforest

[CV]  polynomialfeatures__degree=2, randomforestregressor__bootstrap=False, randomforestregressor__max_depth=30, randomforestregressor__max_features=sqrt, randomforestregressor__min_samples_leaf=4, randomforestregressor__min_samples_split=10, randomforestregressor__n_estimators=1800, score=-7.623844, total=   2.5s
[CV] polynomialfeatures__degree=2, randomforestregressor__bootstrap=False, randomforestregressor__max_depth=50, randomforestregressor__max_features=auto, randomforestregressor__min_samples_leaf=1, randomforestregressor__min_samples_split=2, randomforestregressor__n_estimators=200 
[CV]  polynomialfeatures__degree=2, randomforestregressor__bootstrap=False, randomforestregressor__max_depth=50, randomforestregressor__max_features=auto, randomforestregressor__min_samples_leaf=1, randomforestregressor__min_samples_split=2, randomforestregressor__n_estimators=200, score=-9.932104, total=   0.8s
[CV] polynomialfeatures__degree=2, randomforestregressor__bootstrap=False, randomforestr

In [181]:
random_forest.get_params()

{'bootstrap': True,
 'criterion': 'mse',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'min_impurity_split': 1e-07,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 10,
 'n_jobs': 1,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}